In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [6]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ROEW1HL5Z947HSWMOE1M
%env CLEARML_API_SECRET_KEY=wym7QpDMcGgyGAS2NIHBq53TL6djugqJB5gNgdOUXS1HUtHTYP

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ROEW1HL5Z947HSWMOE1M
env: CLEARML_API_SECRET_KEY=wym7QpDMcGgyGAS2NIHBq53TL6djugqJB5gNgdOUXS1HUtHTYP


In [7]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="drons_vs_berds",
    task_name="detection2"
)
logger = task.get_logger()

ClearML Task: overwriting (reusing) task id=5c5a863934154973ac9b3ed0e0a9255b
2024-04-30 15:07:54,048 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-04-30 15:07:54,071 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/bda991b03ff34114b2ef93fc1e8d32ba/experiments/5c5a863934154973ac9b3ed0e0a9255b/output/log


In [4]:
im = cv2.imread("/home/vlad/datasets/drons_vs_berds/images/2019_10_16_C0003_5043_mavic/0002.jpg")

In [5]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
 

[04/30 13:30:52 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [6]:
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite('./img.jpg', out.get_image()[:, :, ::-1])

/home/vlad/anaconda3/envs/berds_vs_drons/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


True

In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_train.json", "/home/vlad/datasets/drons_vs_berds/images")
register_coco_instances("my_dataset_val", {}, "/home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_valid.json", "/home/vlad/datasets/drons_vs_berds/images")
 

In [9]:
register_coco_instances("my_dataset_test", {}, "/home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_test.json", "/home/vlad/datasets/drons_vs_berds/images")

In [10]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy
import torch

def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [
        T.Resize((1024,1024)),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3),
        T.RandomSaturation(0.8, 1.4),
        T.RandomRotation(angle=[45, 45]),
        T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=False, vertical=True),
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import torch
from detectron2.solver.build import get_default_optimizer_params
from detectron2.solver.build import maybe_add_gradient_clipping

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    # https://medium.com/innovation-res/detectron2-config-optimizer-lr-scheduler-part-1-4555842e1ea
    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Build an optimizer from config.
        """
        params = get_default_optimizer_params(model)
        return maybe_add_gradient_clipping(cfg, torch.optim.AdamW)(
                          params, 
                          lr=cfg.SOLVER.BASE_LR,        
                          weight_decay=cfg.SOLVER.WEIGHT_DECAY)

In [11]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)  

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 3  
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.BASE_LR = 8e-4
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = int(0.2*cfg.SOLVER.MAX_ITER)

# cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.MAX_ITER = 200    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.SOLVER.STEPS = (100, 200)        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
# cfg.INPUT.MIN_SIZE_TRAIN= 800
# cfg.INPUT.MAX_SIZE_TRAIN=  1024
# cfg.INPUT.MIN_SIZE_TEST=  800
# cfg.INPUT.MAX_SIZE_TEST= 1024
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
# trainer.train()

[04/30 15:08:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

In [2]:
import os

In [12]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[04/30 15:08:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
2024-04-30 15:08:26,642 - clearml.model - INFO - Selected model id: ff725052627041a5bc94681af9fd900f


In [48]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
 
result = inference_on_dataset(predictor.model, val_loader, evaluator)

[04/30 17:35:01 d2.data.datasets.coco]: Loaded 7805 images in COCO format from /home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_valid.json
[04/30 17:35:01 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   drone    | 7068         |
|            |              |
[04/30 17:35:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/30 17:35:01 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/30 17:35:01 d2.data.common]: Serializing 7805 elements to byte tensors and concatenating them all ...
[04/30 17:35:01 d2.data.common]: Serialized dataset takes 1.92 MiB
[04/30 17:35:01 d2.evaluation.evaluator]: Start inference on 7805 batches
[04/30 17:35:03 d2.evaluation.evaluator]: Inference done 11/7805. Dataloading: 0.0011 s/iter. Inferen

In [14]:
result

OrderedDict([('bbox',
              {'AP': 4.056669176752632,
               'AP50': 19.234785394932196,
               'AP75': 0.10334012124616718,
               'APs': 4.3205221332592725,
               'APm': 5.843446451718537,
               'APl': 4.369740454518531})])

In [26]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
 
result = inference_on_dataset(predictor.model, val_loader, evaluator)

# for name, value in result['bbox'].items():
#     logger.report_single_value(name=f"test_{name}",value=value)

[04/07 22:49:04 d2.data.datasets.coco]: Loaded 3116 images in COCO format from /home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_test.json


[04/07 22:49:04 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   drone    | 3729         |
|            |              |
[04/07 22:49:04 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/07 22:49:04 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/07 22:49:04 d2.data.common]: Serializing 3116 elements to byte tensors and concatenating them all ...
[04/07 22:49:04 d2.data.common]: Serialized dataset takes 0.81 MiB
[04/07 22:49:04 d2.evaluation.evaluator]: Start inference on 3116 batches
[04/07 22:49:05 d2.evaluation.evaluator]: Inference done 11/3116. Dataloading: 0.0006 s/iter. Inference: 0.0801 s/iter. Eval: 0.0000 s/iter. Total: 0.0807 s/iter. ETA=0:04:10
[04/07 22:49:10 d2.evaluation.evaluator]: Inference done 73/3116. Dataloading: 

In [27]:
result

OrderedDict([('bbox',
              {'AP': 4.086634071732409,
               'AP50': 19.534415589701386,
               'AP75': 0.10334012124616718,
               'APs': 4.3205221332592725,
               'APm': 5.873064462443072,
               'APl': 4.369740454518531})])

In [17]:
import glob
paths = glob.glob("/home/vlad/datasets/drons_vs_berds/images/*/*.jpg")

In [30]:
import random
int(random.random()*10000)

5857

In [49]:
val_loader = build_detection_test_loader(cfg, "my_dataset_val")

[04/30 17:58:23 d2.data.datasets.coco]: Loaded 7805 images in COCO format from /home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_valid.json
[04/30 17:58:23 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/30 17:58:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/30 17:58:23 d2.data.common]: Serializing 7805 elements to byte tensors and concatenating them all ...
[04/30 17:58:23 d2.data.common]: Serialized dataset takes 1.92 MiB


In [43]:
next(iter(val_loader))[0]['file_name'].split('/')[-2]

'00_06_10_to_00_06_27'

In [50]:
path_save = '/home/vlad/projects/drons_vs_berds/train/detection2/valid_predicted/'

for row in val_loader:
    im = cv2.imread(row[0]['file_name'])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    img_path = os.path.join(path_save, f"{row[0]['file_name'].split('/')[-2]}_{os.path.basename(row[0]['file_name'])}")
    cv2.imwrite(img_path, out.get_image()[:, :, ::-1])

KeyboardInterrupt: 

In [20]:
import yaml
    # Save the configuration to a config.yaml file
config_yaml_path = "output/myConfig.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [18]:
outputs

{'instances': Instances(num_instances=13, image_height=2160, image_width=3840, fields=[pred_boxes: Boxes(tensor([[   0.0000,    0.0000,  947.6676,  878.7064],
         [   0.0000,    0.0000, 1880.0162,  862.1476],
         [ 312.2399,    0.0000, 3132.5972,  850.6199],
         [1375.0640,    0.0000, 3840.0000,  848.4592],
         [2616.6394,    0.0000, 3840.0000,  848.7403],
         [3243.2583,    0.0000, 3840.0000,  868.5027],
         [3784.4607,    0.0000, 3840.0000,  874.7025],
         [ 785.5020,    0.0000, 3605.8704, 1273.0992],
         [3511.4888,    0.0000, 3840.0000, 1321.7504],
         [   0.0000,    0.0000, 1841.7069, 1756.2111],
         [1837.1652,    0.0000, 3840.0000, 1734.8438],
         [2953.6809,    0.0000, 3840.0000, 1743.2695],
         [   0.0000,    0.0000, 3504.6045, 2160.0000]], device='cuda:0')), scores: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'), pred_classes: tensor([74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74],

In [10]:
result

OrderedDict([('bbox',
              {'AP': nan,
               'AP50': nan,
               'AP75': nan,
               'APs': nan,
               'APm': nan,
               'APl': nan})])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
 
result = inference_on_dataset(predictor.model, val_loader, evaluator)

for name, value in result['bbox'].items():
    logger.report_single_value(name=f"val_{name}",value=value)

In [13]:
trainer.test(cfg)

TypeError: DefaultTrainer.test() missing 1 required positional argument: 'model'

In [ ]:
cfg 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/home/vlad/projects/drons_vs_berds/train/detection2/output/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3
predictor = DefaultPredictor(cfg)

In [22]:
cfg = get_cfg()

cfg.merge_from_file('/home/vlad/projects/drons_vs_berds/train/detection2/output/myConfig.yaml')

 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
 
cfg.MODEL.WEIGHTS = '/home/vlad/projects/drons_vs_berds/train/detection2/output/model_final.pth'
predictor = DefaultPredictor(cfg)


Loading config /home/vlad/projects/drons_vs_berds/train/detection2/output/myConfig.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[04/07 22:46:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/vlad/projects/drons_vs_berds/train/detection2/output/model_final.pth ...


In [37]:
import glob

In [ ]:
tests_videos=[
    "00_06_10_to_00_06_27",
    "2019_08_19_GP015869_1520_inspire" ,
    "GOPR5843_002",
    "dji_phantom_4_mountain_hover",
    "gopro_002",
    "parrot_disco_zoomin_zoomout",
    "two_distant_phantom"
]

In [62]:
paths = glob.glob('/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/*.jpg')

In [63]:
paths

['/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0184.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0000.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0008.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0296.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0414.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0396.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0422.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0104.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0244.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0258.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0018.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0338.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0378.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR5843_002/0050.jpg',
 '/home/vlad/datasets/drons_vs_berds/images/GOPR

In [65]:
im = cv2.imread(paths[-80])
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite('./img.jpg', out.get_image()[:, :, ::-1])

True

In [31]:
outputs["instances"]

Instances(num_instances=1, image_height=1080, image_width=1920, fields=[pred_boxes: Boxes(tensor([[1059.5670,  543.9017, 1123.1814,  600.6081]], device='cuda:0')), scores: tensor([0.9905], device='cuda:0'), pred_classes: tensor([0], device='cuda:0')])